<h3> Tokenizator OpenAI

https://platform.openai.com/tokenizer

<h3> Generowanie tekstu - znak po znaku (za pomocą GRU)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [3]:
texts = ["Bioinformatyka to interdyscyplinarna dziedzina nauki, która łączy biologię, informatykę i matematykę w celu analizy i interpretacji danych biologicznych."] 
         
print(texts)
print()

# Model: 
class TextGenerator(nn.Module):
    def __init__(self, v_size, hidden_size, n_layers=1):
        super().__init__()
        self.ident = torch.eye(v_size) #macierz wektorow one-hot encoding dla wszytkich znakow
        self.gru = nn.GRU(v_size, hidden_size, n_layers, batch_first=True) #uzyjemy sobie GRU
        self.decoder = nn.Linear(hidden_size, v_size) #jako dekoder wezmiemy przeksztalcenie liniowe
    
    def forward(self, inp, hidden=None):
        inp = self.ident[inp]                  #one-hot vector dla kolejnego znaku
        output, hidden = self.gru(inp, hidden) #zastosowanie GRU
        output = self.decoder(output)          #rozklad kolejnych znakow
        return output, hidden
    

txt = ["<BOS>"] + list(texts[0]) + ["<EOS>"]
vocab = list(set(texts[0])) + ["<BOS>", "<EOS>"]
vocab_size = len(vocab) #zapisuje te informacje pod zmienną
vocab1 = {s: i for i, s in enumerate(vocab)}
vocab2 = {i: s for i, s in enumerate(vocab)}

model = TextGenerator(vocab_size, 20) #buduje model, z wymiarem dla stanu ukrytego = 16
criterion = nn.CrossEntropyLoss() #funkcja kosztu


txt_indices = [vocab1[ch] for ch in txt] #indeksy
txt_tensor = torch.Tensor(txt_indices).long().unsqueeze(0) #przerabiamy na tensor


optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #optymalizator
criterion = nn.CrossEntropyLoss()                          #funkcja kosztu


for it in range(2000):
    optimizer.zero_grad()
    input_seq = txt_tensor[:, :-1]    #wejscie <- bez ostatniego znaku
    target = txt_tensor[:, 1:]        #target <- od 1 znaku
    output, hidden = model(input_seq)
    loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))    
    loss.backward()
    optimizer.step()

    if (it+1) % 100 == 0:
        print(f"[Iter {it+1}] Loss value: {float(loss):.4f}")

['Bioinformatyka to interdyscyplinarna dziedzina nauki, która łączy biologię, informatykę i matematykę w celu analizy i interpretacji danych biologicznych.']

[Iter 100] Loss value: 2.9913
[Iter 200] Loss value: 2.4986
[Iter 300] Loss value: 1.7353
[Iter 400] Loss value: 1.1184
[Iter 500] Loss value: 0.6791
[Iter 600] Loss value: 0.3876
[Iter 700] Loss value: 0.2260
[Iter 800] Loss value: 0.1474
[Iter 900] Loss value: 0.1034
[Iter 1000] Loss value: 0.0770
[Iter 1100] Loss value: 0.0597
[Iter 1200] Loss value: 0.0478
[Iter 1300] Loss value: 0.0393
[Iter 1400] Loss value: 0.0328
[Iter 1500] Loss value: 0.0279
[Iter 1600] Loss value: 0.0240
[Iter 1700] Loss value: 0.0208
[Iter 1800] Loss value: 0.0182
[Iter 1900] Loss value: 0.0161
[Iter 2000] Loss value: 0.0143


In [5]:
import time
from IPython.display import clear_output

def sample_sequence(model, max_len=100, temperature=0.8):
    generated_sequence = "" #tu będzie przechowywana wygenerowana sekwencja
    inp = torch.Tensor([vocab1["<BOS>"]]).long() #zaczynamy od BOS
    hidden = None
    for p in range(max_len):
        output, hidden = model(inp.unsqueeze(0), hidden) #co nam przeiwduje model?
        output_dist = output.data.view(-1).div(temperature).exp() #uwzględniamy czynnik temperaturowy T, zamiast zi mamy zi/T--> normalnie
        top_i = int(torch.multinomial(output_dist, 1)[0]) #indeks przewidywanego znaku
        predicted_char = vocab2[top_i] #przewidywany znak
        if predicted_char == "<EOS>": #jakim przewidywanym znakiem jest EOS to konczymy
            break
        generated_sequence += predicted_char #dodajemy kolejny znak
        inp = torch.Tensor([top_i]).long() #zapisujemy indeks w postaci tensora
        clear_output(wait=True) 
        print(generated_sequence)
        time.sleep(0.1) 
    return "Done"

In [8]:
print(sample_sequence(model, temperature=0.4)) #T=0.4

Bioinformatyka to interdyscyplinarna dziedzina nauki, która łączy biologię, informatykę i matematykę
Done


In [7]:
print(sample_sequence(model, temperature=1.2)) #T=1.2

Bioinforerpretacji danych biobiologicznaczywh.
Done


<h3> Modele Seq2Seq

Tekst -> Tekst

Gdzie?
- Streszczenia
- Tłumaczenia
- Tekst piosenki -> Tytuł
- Czaty, w tym automaty: Pytanie/Odpowiedź
- i wiele innych

<h3> Prosty model do generowania podsumowania (bez sieci neuronowych)

In [45]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from nltk.corpus import stopwords

print("Cały tekst:")
text = "Bioinformatics is an interdisciplinary field that combines biology, computer science, mathematics, and statistics to analyze and interpret biological data. It has become an essential tool in modern biological research, particularly due to the rapid growth of genomic and molecular data generated by new technologies. The origins of bioinformatics can be traced back to the early days of genome sequencing, when scientists needed tools to store, search, and analyze DNA sequences. Today, bioinformatics is applied in a wide range of biological and medical fields, such as genomics, transcriptomics, proteomics, and systems biology. One of the most important applications of bioinformatics is in genomics, where it is used to assemble genomes, identify genes, and study genetic variations between individuals or species. Bioinformatics tools also help predict the structure and function of proteins, model biological pathways, and analyze evolutionary relationships. In medicine, bioinformatics plays a critical role in personalized medicine, where patients' genetic data can be analyzed to tailor drug treatments or predict disease risks. Bioinformatics methods are also used in cancer research, vaccine development, and drug discovery. A key feature of bioinformatics is its reliance on databases, algorithms, and software tools to process vast amounts of biological information. Tools like BLAST (for sequence comparison), multiple sequence alignment algorithms, and molecular modeling software are essential in many bioinformatics workflows. As the amount of biological data continues to grow exponentially, the demand for skilled bioinformaticians is also increasing. The field continues to evolve, incorporating machine learning, artificial intelligence, and big data technologies to gain new insights into complex biological systems. Bioinformatics not only helps scientists understand life at a molecular level but also bridges the gap between experimental data and practical applications in health, agriculture, and biotechnology. It is a dynamic and rapidly expanding discipline that plays a vital role in the future of science and medicine."
print(text)
print("----------------------------------------------------")

stop_words = set(stopwords.words("english"))
sentences = sent_tokenize(text)
words = word_tokenize(text.lower())

#Zliczenia słów
word_freq = Counter(words)

#Wybór "najlepszych" zdań
sentence_scores = {}
for sent in sentences:
    sentence_words = word_tokenize(sent.lower())
    score = [word_freq[word] for word in sentence_words if word.isalnum() and word not in stop_words]
    score = sum(score)/len(score)
    sentence_scores[sent] = score

#wybór top 3 zdań
summary = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:3]

print("Podsumowanie:")
print(" ".join(summary))

Cały tekst:
Bioinformatics is an interdisciplinary field that combines biology, computer science, mathematics, and statistics to analyze and interpret biological data. It has become an essential tool in modern biological research, particularly due to the rapid growth of genomic and molecular data generated by new technologies. The origins of bioinformatics can be traced back to the early days of genome sequencing, when scientists needed tools to store, search, and analyze DNA sequences. Today, bioinformatics is applied in a wide range of biological and medical fields, such as genomics, transcriptomics, proteomics, and systems biology. One of the most important applications of bioinformatics is in genomics, where it is used to assemble genomes, identify genes, and study genetic variations between individuals or species. Bioinformatics tools also help predict the structure and function of proteins, model biological pathways, and analyze evolutionary relationships. In medicine, bioinforma

<h3> Model do tłumaczenia oparty o sieci rekurencyjne (GRU)

In [46]:
# Przykładowe dane
dane = [
    ("Czesc", "Hello"),
    ("Jak sie masz", "How are you"),
    ("Mam na imie Anna", "My name is Anna"),
    ("To jest moj kot", "This is my cat"),
    ("Lubie kawe", "I like coffee"),
    ("Dzisiaj jest zimno", "It is cold today"),
    ("Idziemy do sklepu", "We are going to the store"),
    ("On jest moim bratem", "He is my brother"),
    ("To bardzo dobre", "This is very good"),
    ("Nie rozumiem", "I do not understand"),
    ("Mieszkam w Krakowie", "I live in Cracow"),
    ("Chce sie uczyc angielskiego", "I want to learn English"),
    ("To jest latwe", "This is easy"),
    ("Zaraz wracam", "I will be right back"),
    ("Jestem zmeczony", "I am tired"),
    ("Czy mozesz mi pomoc", "Can you help me"),
    ("Lubie czytac ksiazki", "I like reading books"),
    ("O ktorej godzinie", "At what time"),
    ("Dobrze cie widziec", "Good to see you"),
    ("Nie mam czasu", "I do not have time"),
    ("To jest moj dom", "This is my house"),
    ("Nie wiem", "I do not know"),
    ("Co to znaczy", "What does it mean"),
    ("Uwazaj", "Be careful"),
    ("Jestem glodny", "I am hungry"),
    ("Pojdzmy na spacer", "Let us go for a walk"),
    ("To jest moj przyjaciel", "This is my friend"),
    ("Ucze sie polskiego", "I am learning Polish"),
    ("Potrzebuje pomocy", "I need help"),
    ("To nie ma sensu", "That does not make sense")
]
# Budujemy słownik
def build_vocab(dane):
    vocab = {"<PAD>": 0, "<SOS>":1, "<EOS>":2}
    idx = 3
    for src, tgt in dane:
        for word in word_tokenize(src.lower()) + word_tokenize(tgt.lower()):
            if word not in vocab:
                vocab[word] = idx
                idx += 1
    return vocab

vocab = build_vocab(dane)
inv_vocab = {v: k for k, v in vocab.items()}

def encode(sentence, vocab, sos=False, eos=False):
    ids = [vocab[word] for word in word_tokenize(sentence.lower())]
    if sos: ids = [vocab["<SOS>"]] + ids
    if eos: ids = ids + [vocab["<EOS>"]]
    return torch.tensor(ids)

#dane
pairs = [(encode(src, vocab), encode(tgt, vocab, sos=True, eos=True)) for src, tgt in dane]

#parametry
vocab_size = len(vocab)
embedding_dim = 64
hidden_dim = 128

In [47]:
#Enkoder -> cel: zakodowanie zdania wejsciowego
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        
    def forward(self, x):
        x = self.embed(x)
        _, hidden = self.gru(x)
        return hidden

#Dekoder -> cel: rozkodowanie zdania wyjsciowego
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, hidden):
        x = self.embed(x)
        output, hidden = self.gru(x, hidden)
        output = self.fc(output)
        return output, hidden

encoder = Encoder()
decoder = Decoder()

criterion = nn.CrossEntropyLoss(ignore_index=0)
enc_optim = optim.Adam(encoder.parameters(), lr=0.01)
dec_optim = optim.Adam(decoder.parameters(), lr=0.01)

#Pętla ucząca
for epoch in range(100):
    total_loss = 0
    for src, tgt in pairs:
        src = src.unsqueeze(0)  # batch=1
        tgt_input = tgt[:-1].unsqueeze(0)
        tgt_target = tgt[1:].unsqueeze(0)

        hidden = encoder(src)
        output, _ = decoder(tgt_input, hidden)

        loss = criterion(output.squeeze(0), tgt_target.squeeze(0))
        
        enc_optim.zero_grad()
        dec_optim.zero_grad()
        loss.backward()
        enc_optim.step()
        dec_optim.step()

        total_loss += loss.item()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

Epoch 0, Loss: 127.80132937431335
Epoch 10, Loss: 0.19135316263418645
Epoch 20, Loss: 0.06985790480393916
Epoch 30, Loss: 0.03791447635740042
Epoch 40, Loss: 0.023940994375152513
Epoch 50, Loss: 0.016431700496468693
Epoch 60, Loss: 0.01189034795970656
Epoch 70, Loss: 0.00892099273914937
Epoch 80, Loss: 0.00687036981980782
Epoch 90, Loss: 0.00539706524432404


* Czy to działa?

In [54]:
#Czy to działa?
def generate(tekst):
    src = encode(tekst, vocab).unsqueeze(0)
    hidden = encoder(src)
    
    input_token = torch.tensor([[vocab["<SOS>"]]])
    result = []

    for _ in range(15):
        output, hidden = decoder(input_token, hidden)
        pred_id = output.argmax(2).item() #wybieram najbardziej prawdopodobne slowo
        if pred_id == vocab["<EOS>"]:
            break
        result.append(inv_vocab[pred_id])
        input_token = torch.tensor([[pred_id]])
        clear_output(wait=True) 
        print(" ".join(result))
        time.sleep(0.3) 
    return "Done"

In [55]:
print(generate("Zaraz wracam"))

i will be right back
Done


Usprawnienia:
- rozbudowa sieci
- mechanizm atencji
- teacher forcing

Warto poczytać: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

<h4> Jak ocenić wygenerowany tekst?
    
- ROUGE (najczęściej przy streszczeniach,  oparty o n-gramy)
- BLEU (najczęściej do tłumaczeń, dosyć restrykcyjny, oparty o n-gramy)
    
Istnieją też bardziej zaawansowane miary, jak BERTScore, ten ocenia semantyczne podobieństwo odpowiedzi z użyciem embeddingów.

https://lightning.ai/docs/torchmetrics/stable/text/rouge_score.html

https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363

In [47]:
from rouge_score import rouge_scorer

# Przykładowe teksty
reference = "to jest sa przykladowe zdanie"
candidate = "przykladowe to jest zdania"

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True) #L - najdluzszy podciąg
scores = scorer.score(reference, candidate)
print(scores)

{'rouge1': Score(precision=0.75, recall=0.6, fmeasure=0.6666666666666665), 'rouge2': Score(precision=0.3333333333333333, recall=0.25, fmeasure=0.28571428571428575), 'rougeL': Score(precision=0.5, recall=0.4, fmeasure=0.4444444444444445)}


In [48]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

reference = [["to", "jest", "przykładowe", "zdanie"]]
candidate = ["to", "jest", "przykład", "zdania"]

# Obliczanie BLEU
bleu_score = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1) #(ile z wygenerowanych n-gramow znajduje sie w referencji)
print("BLEU score:", bleu_score)

BLEU score: 0.16990442448471224


- Na następne zajęcia proszę wykonać jedno z poniższych zadań (do wyboru). W razie konieczności zmniejsz rozważany zbiór danych (dotyczy szczególnie zad2). Na Teams proszę zamieścić plik jupyter oraz pdf.

<h4> Zadanie1. Zbuduj (zgodnie z tutorialem) model do tłumaczenia tekstu z języka francuskiego na angielski. 
    
- Dodatkowo, podziel na początku zbiór danych na treningowy, testowy, walidacyjny (zaproponuj w jakich proporcjach)
- Zastosuj metrykę BLEU do tego zadania jako miarę oceniającą wygenerowane tłumaczenia
- Potestuj kilka architektur sieci neuronowych, opcjonalnie możesz wykorzystać również GloVe
- Opracuj w formie krótkiego sprawozdania (max 2 strony) najważniejsze kroki analizy oraz zamieść odpowiednie wykresy - spadek funkcji kosztu w kolejnych epokach (na zbiorze treningowym i walidacyjnym), wizualizacje atencji na przykładowych zdaniach, miara BLEU na zbiorze treningowym, walidacyjnym i testowym. Ile danych rozważano?
    
    
https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

<h4> Zadanie2: Dokonaj dostrojenia modelu T5 (Text-to-Text Transfer Transformer) do zadania generowania podsumowań na zbiorze

https://huggingface.co/datasets/knkarthick/samsum

In [14]:
import pandas as pd
splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/knkarthick/samsum/" + splits["train"])

<h4>

- Opisz krótko model T5 (co to za model, jaka architektura, ile parametrów, na jakich tekstach trenowano)
- Czego dotyczą te dane?
- Porównaj słowa występujące w streszczeniu i pełnym tekście (w szczególności, jaki % ich się znajduje w pełnym tekście)
- Narysuj rozkład długości tekstów pełnych i streszczeń
- Rozważ tylko te najkrótsze teksty, jakie?
- Zastosuj metrykę ROUGE do oceny modelu: przed treningiem i po (na rozważanych zbiorach)
- Potestuj różne hiperparametry (szczególnie w TrainingArguments)
- W krótkim sprawozdaniu (max 2 strony) opisz dokładnie wszystkie argumenty występujące w TrainingArguments oraz Trainer oraz zamieść wykresy funkcji kosztu podczas procesu uczenia oraz wyniki dotyczące metryki ROUGE.

Możesz oprzeć się na:
    
* https://huggingface.co/google-t5/t5-small
* https://medium.com/@anyuanay/fine-tuning-the-pre-trained-t5-small-model-in-hugging-face-for-text-summarization-3d48eb3c4360
* https://medium.com/@syahmisajid12/summarization-using-fine-tuned-t5-transformer-62d0630ff795  

T5 (Text-to-Text Transfer Transformer) 

<h3> To się może przydać (np do projektu)

*  ponad 5000 par pytanie–odpowiedź w języku polskim,

https://www.futurebeeai.com/dataset/prompt-response-dataset/polish-closed-ended-question-answer-text-dataset

* 7000 pytań i inne

https://huggingface.co/datasets/ipipan/polqa

- streszczenia

https://huggingface.co/datasets/allegro/summarization-polish-summaries-corpus

- posiedzenia sejmu

https://www.sejm.gov.pl/sejm7.nsf/poslowie.xsp?type=P#K

Mamy też Web Scraping, ale pamiętajmy! 
- jeżeli to są małe ilości, krótkie wypowiedzi (prawo cytatu) to raczej ok
- zazwyczaj nie ma problemu ze źródłami z domen publicznych
- w przypadku stron komercyjnych, duże ilości pobieranych danych -> często udostępniane są API (to może być płatne)
- jeżeli byśmy chcieli wykorzystywać zgromadzone dane czy zbudowane modele komercyjne, to sytucja bardziej skomplikowana :)

<h3> Inne, warte rozważenia zbiory danych

https://github.com/allenai/gooaq #pytania/odp na podstawie sugestii google

https://rajpurkar.github.io/SQuAD-explorer/

<h3> Speakleash (spichlerz)

https://pypi.org/project/speakleash/

https://github.com/speakleash/speakleash

- narzędzie w Pythonie służące do zarządzania dużymi zbiorami danych tekstowych w języku polskim, wykorzystywane m.in. do trenowania modeli językowych
- obszerny zestaw metryk + klasyfikacja tematyczna dokumentu

<h3> Bielik

- duży polskojęzyczny model językowy (LLM), miliardy parametrów
- wytrenowany na danych z projektu SpeakLeash
- oparty na architekturze Transformers

https://bielik.ai/

https://huggingface.co/spaces/speakleash/Bielik-7B-Instruct-v0.1 (starsza wersja)

https://huggingface.co/speakleash/Bielik-11B-v2